In [4]:
import daw_readout
import process_data
import pandas as pd
import numpy as np
import h5py
from tqdm import tqdm
import sys
import time
import os

In [5]:
#file_list = 'runlist/darkrate_monitor_with_GECO2020'
#file_list = 'runlist/LED_1.7V_11ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_7.5ns_0.5_ratio_runs'
#file_list = 'runlist/LED_1.7V_7.5ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_10ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_11ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_12ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_13ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_14ns_400_ratio_runs'
#file_list = 'runlist/LED_1.7V_15ns_400_ratio_runs'

file_list = 'runlist/LED_20240826_1.13V_63ns'

In [6]:
path_save =""
winfo =[]
with open(file_list, 'r') as list:
    for line in list:    
        rawfilename = line.rstrip('\n')[17 :] 
        rawdata = daw_readout.DAWDemoWaveParser(rawfilename)     
        for wave in tqdm(rawdata) :            
            ch = wave.Channel
            ttt = wave.Timestamp
            base = wave.Baseline
            pulse = wave.Waveform
            st, minp, ed = process_data.pusle_index(pulse)
            ht = base - wave.Waveform[minp]            
            area = process_data.pusle_area(pulse, st, ed, base)
            area_fix_len = process_data.pulse_area_fix_len(pulse, minp, 15, base)
            area_fix_range = process_data.pulse_area_fix_range(pulse, 95, 130, base)
            area_fix_range_dy = process_data.pulse_area_fix_range(pulse, 90, 150, base)
            if ch == 0:
                area_pe = area / 77.791
                area_fix_len_pe = area_fix_len/ 77.791
                area_fix_range_pe = area_fix_range / 77.791
            if ch == 1:
                area_pe = area / 96.99
                area_fix_len_pe = area_fix_len/ 96.99
                area_fix_range_pe = area_fix_range / 96.99
            if ch == 2:
                area_pe = area / 77.791
                area_fix_len_pe = area_fix_len / 77.791
                area_fix_range_pe = area_fix_range_dy / 77.791 
            winfo.append({         
                #'EvtID': i,
                'Ch':ch,
                'TTT':ttt,   ## Trigger time tag
                'Baseline': base, 
                'Hight': ht, 
                'St': st,
                'Ed': ed,
                'Minp': minp,
                'Width': ed - st,  
                'Area': area_pe,     ## area of pulse cal by dynamic range
                'Area_fixlen': area_fix_len_pe,
                'Area_fixrange':area_fix_range_pe,
                'Wave': pulse
            })            
        file_tag = line.rstrip('\n')[17 :].rstrip('.bin')[24:][: -12]  
        path_save = "outnpy/{}_debug_area.h5df".format(file_tag)
        df = pd.DataFrame(winfo)
        process_data.write_to_hdf5(df, path_save)

360828it [00:14, 24471.47it/s]


h5 Write Time: 1.65 s 
h5 File Size: 123.27 MB
Save to outnpy/LV2415_anodereadout_LV2414_dualreadout_20240826_LED_1.13V_63ns_400_ratio_run0_debug_area.h5df


In [ ]:
#df = pd.DataFrame(winfo)
#data_array = df.values
#np.save(path_save, data_array)